In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Initial Analysis

In [ ]:
df = pd.read_csv('/kaggle/input/yeh-concret-data/Concrete_Data_Yeh.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()

# 2. EDA (univariate)

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(df["age"])

In [ ]:
sns.color_palette("pastel")
for col in df.columns:
    if col !='age':
        sns.histplot(df[col],kde=True, color='yellow')
        plt.show()

## Not in Guassian form : cement, slag, flyash, superplasticizer (Skewed Data)

In [ ]:
df.head()

In [ ]:
sns.set_style("whitegrid")
sns.color_palette("pastel")
for col in df.columns:
    if col !='age':
        sns.boxplot(df[col])
        plt.show()

## Some outliers in columns. If there are a lot, we can fix them. 

# 3. EDA (bivariate) 

In [ ]:
plt.figure(figsize=(15,15))
sns.pairplot(df)





## Here, we can find the correlation between the variables 

# 4. Feature Selection 

## Heatmap

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(),annot=True,cmap='Spectral')

In [ ]:
X = df.drop(['csMPa'],axis=1)
y = df["csMPa"]

##  Extra Trees Regressor 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, StackingRegressor, BaggingRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
feature_imps = pd.Series(model.feature_importances_, index = X.columns)
feature_imps.nlargest(8).plot(kind='barh')

## Features ranked based on importance 

## Number of features to select

In [ ]:
from sklearn.decomposition import PCA
plt.figure(figsize=(7,5))
pca_dummy = PCA(n_components=None)
X_dummy = X
X_dummy = pca_dummy.fit_transform(X_dummy)
plt.plot(np.cumsum(pca_dummy.explained_variance_ratio_))

### We have to select more than 4 features for best results 

# 5. Feature scaling

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size = 0.2, random_state=15)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 6. Model Building

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
score = r2_score(y_test,y_pred)
score
print('Accuracy of the Stacking regressor model is {}'.format(score*100))

In [ ]:
print(cross_val_score(xgb, X_train, y_train, cv=5, scoring = 'r2').mean())

In [ ]:
cross_val_score(xgb, X_test, y_test, cv=5, scoring = 'r2').mean()

In [ ]:
from sklearn.linear_model import LinearRegression, SGDRegressor, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, StackingRegressor, BaggingRegressor
from sklearn.metrics import r2_score

In [ ]:
et = ElasticNet()
gb = GradientBoostingRegressor()
sr = StackingRegressor(estimators=[
    ('et', et),
    ('gb', gb),
    ('xgb',xgb)])

sr.fit(X_train,y_train)



In [ ]:
y_pred = sr.predict(X_test)
score = r2_score(y_test,y_pred)
print('Accuracy of the Stacking regressor model is {}'.format(score*100))

In [ ]:
cross_val_score(sr, X_train, y_train, cv=5, scoring = 'r2').mean()

In [ ]:
cross_val_score(sr, X_test, y_test, cv=5, scoring = 'r2').mean()